In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init


In [ ]:
class PreNorm(nn.Module): # f^pre
    def __init__(self, dim : int, fn): # dim : l, fn : z^l -(fn)-> z^(l+1)
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)
    
class FeedForward(nn.Module):
    def __init__(self, dim : int, hidden_dim : int, step_size : float = 0.1, droput = 0.):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(dim, dim))
        with torch.no_grad: 
            nn.init.kaiming_uniform_(self.weight)
        self.step_size = step_size
        self.lmbda = 0.1

    def forward(self, x, **kwargs):
        Dl = nn.Linear(x, self.weight)
        Dl_1 = nn.Linear(Dl, self.weight.t())
        D2 = nn.Linear(x, self.weight.t())

        return self.step_size(Dl_1 - Dl) * (self.lmbda * self.step_size)

            
class Attention(nn.Module):
    

